# Memorabel Personal Health Train Demo

### Settings

In [169]:
from vantage6.client import Client
from pathlib import Path
import time
import requests
import json
import pickle

In [170]:
url_base = "https://dev.v6.personalhealthtrain.net"
api_path = "/api"
url = url_base + api_path

username = "inigo"
password = "WelcomeToVantage6"

### Authentication

In [171]:
print("Attempt login to Vantage6 API")
client = Client(url_base, 443, "/api")

Attempt login to Vantage6 API


In [172]:
client.authenticate(username, password)

In [173]:
client.setup_encryption(None)

In [190]:
json_data = {
  "username": username,
  "password": password
}

# make an http-call to obtain a JWT-token
response = requests.post(
  f"{url_base}{api_path}/token/user", 
  json=json_data
)

# extract JWT access token
data = response.json()
access_token = data["access_token"]
refresh_token = data["refresh_token"]

### Submit task

In [68]:
input_ = {
    "method":"summary", 
    "args": [
        {
            "Age":"Int64",
            "Sex":"category", 
            "MMSE Total Score":"Int64", 
            "CRP":"Int64", 
            "Amyloid Beta":"Int64", 
            "Total Tau":"Int64", 
            "ICV": "Int64",
            "WMH Fazekas score": "Int64"}, 
        ".",
        ";"], 
    "kwargs": {}
}

In [76]:
task = client.post_task(
    name="testing",
    image="harbor.vantage6.ai/algorithms/summary",
    collaboration_id=2,
    input_= input_,
    organization_ids=[2]
)

In [187]:
input_ = {}
task = client.post_task(
    name="testing",
    image="docker.io/ibermejo/sha256_test@sha256:a512144a9741d5495ac2012b859dabeb04a9829e06d41b151b709e905e661613",
    collaboration_id=2,
    input_= input_,
    organization_ids=[3]
)

### Read results

In [188]:
res = client.get_results(task_id=task["id"])

In [189]:
print(res[0])

{'started_at': '2020-09-09T09:34:06.590409+00:00', 'organization': {'id': 3, 'link': '/api/organization/3', 'methods': ['GET', 'PATCH']}, 'log': None, 'assigned_at': '2020-09-09T09:34:06.372416+00:00', 'id': 65, 'task': {'id': 38, 'link': '/api/task/38', 'methods': ['GET', 'DELETE']}, 'input': b'\x80\x03}q\x00.', 'finished_at': None, 'result': None}


In [45]:
from time import sleep

# HTTP headers for authorization
headers = {'Authorization': 'Bearer ' + access_token}

task_id = task.get("id")
task_info = requests.get(
  f"{url_base}{api_path}/task/{task_id}", 
  headers=headers
)

In [229]:
# HTTP headers for authorization
headers = {'Authorization': 'Bearer ' + access_token}

result = requests.get(
  f"{url_base}{api_path}/result/82", 
  headers=headers
)

print(result.json())

{'started_at': '2020-09-09T14:22:50.428064+00:00', 'organization': {'id': 3, 'link': '/api/organization/3', 'methods': ['GET', 'PATCH']}, 'log': "-----------------------------------------\n  Welcome to the Vantage Infrastructure\n-----------------------------------------\nUsing '/mnt/data/database.csv' as database \nLoading data from '/mnt/data/task-000000082/input' \nLoading token from '/mnt/data/task-000000082/token' \nDispatching ... \nRunning a (mere) regular container. \nCalling RPC_node_beta \nWriting data to '/mnt/data/task-000000082/output' \n \n[DONE!] \n", 'assigned_at': '2020-09-09T13:42:40.688320+00:00', 'id': 82, 'task': {'id': 44, 'link': '/api/task/44', 'methods': ['GET', 'DELETE']}, 'input': 'WAoAAAADAAMGAQADBQAAAAAGQ1AxMjUyAAACEwAAAAUAAAAKAAAAAQAAAAAAAAAKAAAAAQAAAAAAAAAQAAAAAQAEAAkAAAAJbm9kZV9iZXRhAAACEwAAAAAAAAQCAAAAAQAEAAkAAAAFbmFtZXMAAAAQAAAAAAAAAP4AAAITAAAAAQAAAhMAAAAFAAADBgAABAIAAAABAAQACQAAAAVjbGFzcwAAABAAAAABAAQACQAAAAdmb3JtdWxhAAAEAgAAAAEABAAJAAAADC5FbnZpcm9ubW

In [197]:
enc_result = result.json()['result']

In [211]:
# HTTP headers for authorization
headers = {'Authorization': 'Bearer ' + access_token}

#while not task.get("complete"):
#requests.delete(
#  f"{url_base}{api_path}/collaboration/2",
#  headers=headers
#)
  #sleep(1)

<Response [200]>

In [54]:
print(task_info)

<Response [500]>


In [195]:
from vantage6.client.encryption import CryptorBase, RSACryptor, DummyCryptor

In [196]:
cryptor = DummyCryptor()

In [199]:
print(pickle.loads(cryptor.decrypt_str_to_bytes(enc_result)))

UnpicklingError: invalid load key, '\x00'.